In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/joe/Documents/boulot/git/homhots/HOTS/HOTS


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import pickle
from mix_Network import *
from Event import Event
from Tools import LoadObject, EuclidianNorm, NormalizedNorm, BattachaNorm

# Defining the network and training

In [ ]:
hots = network(algo='lagorce', krnlinit='first', tau=0.00002)
hots.learninglagorce(diginit=True)

 43%|████▎     | 97044/223371 [00:35<00:51, 2461.15it/s]

In [ ]:
hots.plotlayer()

# Running the Network on training samples

In [ ]:
trainmap = hots.traininglagorce()

# Runing the Network on testing samples

In [ ]:
testmap = hots.testinglagorce()

# Classifying

In [ ]:
def EuclidianNorm(hist1,hist2):
    return np.linalg.norm(hist1-hist2)

def NormalizedNorm(hist1,hist2):
    hist1/=np.sum(hist1)
    hist2/=np.sum(hist2)
    return np.linalg.norm(hist1-hist2)/(np.linalg.norm(hist1)*np.linalg.norm(hist2))

def BattachaNorm(hist1, hist2):
    hist1/=np.sum(hist1)
    hist2/=np.sum(hist2)
    return -np.log(np.sum(np.sqrt(hist1*hist2)))

def accuracy(trainmap,testmap,measure):
    accuracy=0

    for i in range(len(testmap)):
        dist = np.zeros([len(trainmap)])
        for k in range(len(trainmap)):
            if measure=='bhatta':
                dist[k] = BattachaNorm(testmap[i][1],trainmap[k][1])
            elif measure=='eucli':
                dist[k] = EuclidianNorm(testmap[i][1],trainmap[k][1])
            elif measure=='norm':
                dist[k] = NormalizedNorm(testmap[i][1],trainmap[k][1])
        if testmap[i][0]==trainmap[np.argmin(dist)][0]:
            accuracy+=1
    return accuracy

In [ ]:
score1=accuracy(trainmap,testmap,'bhatta')
score2=accuracy(trainmap,testmap,'eucli')
score3=accuracy(trainmap,testmap,'norm')
print('bhatta:'+str(score1/0.4)+'% - '+'eucli:'+str(score2/0.4)+'% - '+'norm:'+str(score3/0.4)+'%')

In [ ]:
fname = "../Records/EXP_01_LagorceKmeans/2020-11-26-mix_hots_validation.pkl"
with open(fname, 'wb') as file:
    pickle.dump([hots, trainmap, testmap], file, pickle.HIGHEST_PROTOCOL)